In [ ]:
!pip install swig   # bridge between low-level C/C++ code and high leve Python code
!pip install gym[box2d] pyglet==1.3.2    # OpenAI gym with Box2D support
                                         # pyglet for rendering 2D graphics
!pip install pygame                      # handling 2D graphics, user input, sound, game development
!pip install -U colabgymrender           # for rendering gym enviroments with Colab
!pip install stable-baselines3[extra]    # reinforcement learning algorithms
!pip install renderlab                   # for rendering enviroment
!pip install pyvirtualdisplay            # simulates displat in headless env (Colab)
!apt-get install -y xvfb ffmpeg          # display server for simulating display
!pip install PyOpenGL PyOpenGL_accelerate# Accelerating rendering
!apt-get install -y xvfb x11-utils       # for display server
!pip install pyvirtualdisplay gymnasium

from pyvirtualdisplay import Display
from IPython.display import HTML
from base64 import b64encode
import gymnasium as gym
from gym.wrappers import RecordVideo
from stable_baselines3.ppo.policies import MlpPolicy
from stable_baselines3 import DQN
from stable_baselines3 import PPO, SAC, TD3
from stable_baselines3.common.vec_env import VecFrameStack
from stable_baselines3.common.evaluation import evaluate_policy
import os
import renderlab as rl
from stable_baselines3.common.vec_env import DummyVecEnv
import time


In [ ]:
display = Display(visible=False, size=(1400, 900))
display.start()

enviroment_name = "LunarLander-v2"
env = gym.make(enviroment_name, render_mode="rgb_array")
env = EnvCompatibility(env)
env = RecordVideo(env, video_folder='./video', episode_trigger=lambda e:True)

def render_video(video_path):
  video = open(video_path, 'rb').read()
  video_encoded = b64encode(video).decode()
  return HTML(f'<video width=400 controls><source src="data:video/mp4;base64,{video_encoded}" type="video/mp4"></video>')



episodes = 2
for episode in range(1, episodes + 1):
    state, info = env.reset()
    done = False
    score = 0

    while not done:
      env.render()
      action = env.action_space.sample()
      n_state, reward, done, info = env.step(action)
      score += reward
    print('Episode:{} Score:{}'.format(episode, score))
env.close()

render_video('./video/rl-video-episode-0.mp4')



In [ ]:
env = gym.make("LunarLander-v2", render_mode="rgb_array")
env = DummyVecEnv([lambda: env])
model = PPO('MlpPolicy', env, verbose = 1)
model.learn(total_timesteps=200000)
LL_path = os.path.join('Training', 'Saved Models', 'PPO_LunarLander_Model')
model.save(LL_path)
model

In [ ]:
def render_video(video_path):
    video = open(video_path, 'rb').read()
    video_encoded = b64encode(video).decode()
    return HTML(f'<video width=400 controls><source src="data:video/mp4;base64,{video_encoded}" type="video/mp4"></video>')

display = Display(visible=False, size=(1400, 900))
display.start()

#env = gym.make("CarRacing-v2", render_mode = "rgb_array")
env = gym.make("LunarLander-v2", render_mode="rgb_array")
env = RecordVideo(env, './video', episode_trigger=lambda e: True)

n_seconds = 10
obs, info = env.reset()
done = False
start_time = time.time()
while not done:
  action, _ = model.predict(obs, deterministic=True)
  obs, reward, done, info = env.step(action)
  env.render()
  if time.time() - start_time > n_seconds:
    break

env.close()

render_video('./video/rl-video-episode-0.mp4')


